In [ ]:
# if viewing to learn. you will want an existing zip folder on .png masks. Look at my other work for help on this!
!unzip -o masks_donut.zip -d masks_donut

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# input and output
mask_dir = "/content/masks_donut/masks_donut"
out_dir  = "masks_donut_gradient"
os.makedirs(out_dir, exist_ok=True)

# loop over masks ensuring each is a png
for fname in sorted(os.listdir(mask_dir)):
    if not fname.endswith(".png"):
        continue

    path = os.path.join(mask_dir, fname)
    # read as grayscale
    mask = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

    # push anything below 127 pixel value to 0, anything above 127 to 255, now binary
    _, mask_bin = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

    # performing a complete dist transform (0 at end means check everywhere in every image for nearest batter edge)
    # typically this would be 3 or 5 i.e. "check a 5x5 box to find the nearest edge" but since colab has computational power I'll run it through everything
    dist = cv2.distanceTransform(mask_bin, cv2.DIST_L2, 0)

    # wanted a slightly more gradual gradient
    dist_norm = np.power(dist, 0.5)

    # map values from min-max to 0-255
    dist_vis = cv2.normalize(dist_norm, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # using magma color code since I like the dark edges against white background
    colored = cv2.applyColorMap(dist_vis, cv2.COLORMAP_MAGMA)
    # set background to white
    colored[mask_bin == 0] = (255, 255, 255)

    out_path = os.path.join(out_dir, fname.replace(".png", "_grad.png"))
    cv2.imwrite(out_path, colored)
